In [5]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import logsumexp
import random
import math
from scipy.stats import multivariate_normal
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import pdist, squareform
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.cluster import KMeans


In [17]:
train = np.array(pd.read_csv('./codon_usage.csv'))
# shuffle the dataset
train = np.delete(train, (486,5063), axis=0)
y = np.copy(train[:,0])
X = np.delete(train,(0,1,2,3,4), axis=1)
X = X.astype(np.float32)

c:\Users\Micha\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

In [11]:
def one_hot_encoding(data):
    num_count = set()
    for i in data:
        num_count.add(i)
    num_count = list(num_count)
    data = np.copy(data)
    zeros = np.zeros((data.shape[0],len(num_count)))
    for i in range(data.shape[0]):
        zeros[i,num_count.index(data[i])] = 1
    return zeros

In [8]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
sss.get_n_splits(X, y)

5

In [89]:
def spectral_clustering(training_data, k, spread, obj):
    data_size = len(training_data)

    # Gaussian Kernel
    norm = squareform(pdist(training_data, 'euclidean'))
    kernel = np.exp(-(norm ** 2)/(2*spread))

    degree_matrix = np.sum(kernel, axis=0) * np.identity(data_size)
    L = np.zeros((data_size, data_size))

    if obj == 'ratio':
        L = degree_matrix - kernel
    elif obj == 'asymmetric':
        L = np.dot(np.linalg.inv(degree_matrix), degree_matrix - kernel)
    elif obj == 'symmetric':
        degree_matrix = np.diag(np.sqrt(np.diag(degree_matrix)))
        degree_matrix = np.linalg.inv(degree_matrix)
        L = np.dot(np.dot(degree_matrix, degree_matrix - kernel), degree_matrix)
    
    _, eigh = np.linalg.eigh(L)
    eigh = eigh[:, 0: k]
    # eigh = np.fliplr(eigh)

    norms = np.linalg.norm(eigh, ord=2, axis=1)
    for i in range(len(norms)):
        eigh[i] = eigh[i, :] / norms[i]

    kmeans = KMeans(n_clusters = k).fit(eigh)

    return kmeans.labels_


def f_measure(clustering_labels, training_target, k):
    training_target = list(np.int_(training_target))
    table = np.zeros((k, max(training_target)+1) , dtype=int)
    # accuracy table calculation
    for target_index, prediction_index in zip(training_target, clustering_labels):
        table[prediction_index][target_index] += 1
    columnwise_sum = np.sum(table, axis=0, dtype=int)
    rowwise_sum = np.sum(table, axis=1, dtype=int)
    precision_vals = []
    recall_vals = []
    for i, row in enumerate(table):
        precision_vals.append(max(row) / rowwise_sum[i])
        x = row.tolist().index(max(row))
        recall_vals.append(max(row) / columnwise_sum[x])
    F_vals = []
    for i in range(k):
        F_vals.append((2 * precision_vals[i] * recall_vals[i]) / (precision_vals[i] + recall_vals[i]))
    return np.mean(F_vals)

def f_measure_with_correspondence(y_true, y_pred, k):
    # calculate F1 score for each class
    for c in range(k):
        # calculate TP, FP, FN for class c
        TP = sum([1 for i, j in zip(y_true, y_pred) if i == c and j == c])
        FP = sum([1 for i, j in zip(y_true, y_pred) if i != c and j == c])
        FN = sum([1 for i, j in zip(y_true, y_pred) if i == c and j != c])
        
    # calculate precision
        precision = TP / (TP + FP)
        
        # calculate recall
        recall = TP / (TP + FN)
        
        # calculate F1 score
        f1 = 2 * (precision * recall) / (precision + recall)
        
        print('F1 score for class', c, ':', f1)
        
def num_of_each_cluster(clustering_labels, k):
    count_each_cluster_size = np.zeros(k)
    for l in range(k):
        count = np.count_nonzero(clustering_labels == l)
        count_each_cluster_size[l] = count

    return count_each_cluster_size


In [95]:
cluster_sizes = []
selected_index = next(sss.split(X,y))[1]
training_data = (X[selected_index])
label = np.argmax(one_hot_encoding(y),axis = 1)[selected_index]
pred = spectral_clustering(training_data, 11, 10, 'asymmetric')
cluster_sizes.append(num_of_each_cluster(pred, 11))
F = f_measure(pred, label, 11)

In [96]:
F

0.32270254399349985

In [97]:
cluster_sizes[0].astype(np.int0)

array([273, 267, 178, 249, 199, 237, 260, 248, 243, 186, 266], dtype=int64)

In [98]:
contigency_table = np.zeros((11,11),dtype = np.integer)
for i in range(len(pred)):
    contigency_table[label[i]][pred[i]] += 1
col_sum = np.sum(contigency_table,axis = 0)
row_sum = np.sum(contigency_table,axis = 1)
H_T = -np.sum(col_sum/np.sum(col_sum)*np.log(col_sum/np.sum(col_sum)))
H_C = -np.sum(row_sum/np.sum(row_sum)*np.log(row_sum/np.sum(row_sum)))
I_CT = 0
for i in range(11):
    for j in range(11):
        if (contigency_table[i][j] != 0):
            I_CT += (contigency_table[i][j]/np.sum(row_sum))*np.log((contigency_table[i][j]*np.sum(row_sum))/((row_sum[i])*(col_sum[j])))
NMI = I_CT/(np.sqrt(H_C*H_T))
NMI

c:\Users\Micha\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Converting `np.integer` or `np.signedinteger` to a dtype is deprecated. The current result is `np.dtype(np.int_)` which is not strictly correct. Note that the result depends on the system. To ensure stable results use may want to use `np.int64` or `np.int32`.
  """Entry point for launching an IPython kernel.


0.26913569565330675